In [8]:
from src.extract.spacex_api import SpaceXExtractor
extractor = SpaceXExtractor()

In [9]:
# Por exemplo, endpoint 'launches'
data = extractor.fetch("rockets")

2026-02-22 15:09:24 [info     ] Iniciando extração de API      endpoint=rockets url=https://api.spacexdata.com/v4/rockets
2026-02-22 15:09:25 [info     ] Extração concluída com sucesso count=4 endpoint=rockets


In [10]:
import json
from IPython.display import display, JSON

# Mostrar os 5 primeiros registros
display(JSON(data[:5]))  # renderiza JSON interativo

<IPython.core.display.JSON object>

In [11]:
all_keys = set()
for item in data:
    all_keys.update(item.keys())

print("Campos encontrados na API:", sorted(all_keys))

Campos encontrados na API: ['active', 'cost_per_launch', 'id', 'name', 'success_rate_pct']


In [13]:
for key in sorted(all_keys):
    example_value = None
    # Procura o primeiro valor não-nulo para cada coluna
    for item in data:
        if key in item and item[key] is not None:
            example_value = item[key]
            break
    print(f"{key}: {example_value}")

active: False
cost_per_launch: 6700000
id: 5e9d0d95eda69955f709d1eb
name: Falcon 1
success_rate_pct: 40.0


In [12]:
import polars as pl

df = pl.from_dicts(data)
df.head()        # mostra as primeiras linhas
df.columns       # mostra todas as colunas detectadas
df.dtypes        # mostra tipos detectados automaticamente

[String, String, Boolean, Int64, Float64]

Table "public.silver_launches"
    Column     |            Type             | Collation | Nullable |      Default      
---------------+-----------------------------+-----------+----------+-------------------
 launch_id     | text                        |           | not null | 
 name          | text                        |           | not null | 
 date_utc      | timestamp without time zone |           |          | 
 rocket        | text                        |           |          | 
 success       | boolean                     |           |          | 
 flight_number | integer                     |           |          | 
 ingested_at   | timestamp without time zone |           |          | CURRENT_TIMESTAMP
Indexes:
    "silver_launches_pkey" PRIMARY KEY, btree (launch_id)
    "idx_silver_launches_date" btree (date_utc)
    "idx_silver_launches_rocket" btree (rocket)
Foreign-key constraints:
    "silver_launches_rocket_fkey" FOREIGN KEY (rocket) REFERENCES silver_rockets(rocket_id)

spacex_db=# 

 Table "public.silver_rockets"
      Column      |            Type             | Collation | Nullable |      Default      
------------------+-----------------------------+-----------+----------+-------------------
 rocket_id        | text                        |           | not null | 
 name             | text                        |           | not null | 
 active           | boolean                     |           |          | 
 cost_per_launch  | bigint                      |           |          | 
 success_rate_pct | double precision            |           |          | 
 ingested_at      | timestamp without time zone |           |          | CURRENT_TIMESTAMP
Indexes:
    "silver_rockets_pkey" PRIMARY KEY, btree (rocket_id)
Referenced by:
    TABLE "silver_launches" CONSTRAINT "silver_launches_rocket_fkey" FOREIGN KEY (rocket) REFERENCES silver_rockets(rocket_id)

spacex_db=# 


Table "public.bronze_rockets"
   Column    |            Type             | Collation | Nullable |      Default      
-------------+-----------------------------+-----------+----------+-------------------
 source      | text                        |           | not null | 
 raw_data    | jsonb                       |           | not null | 
 ingested_at | timestamp without time zone |           |          | CURRENT_TIMESTAMP

spacex_db=# 

Table "public.bronze_launches"
   Column    |            Type             | Collation | Nullable |      Default      
-------------+-----------------------------+-----------+----------+-------------------
 source      | text                        |           | not null | 
 raw_data    | jsonb                       |           | not null | 
 ingested_at | timestamp without time zone |           |          | CURRENT_TIMESTAMP

spacex_db=# 